# Data Cleaning Listado de Listado de Beneficiarios 2019

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. beneficiarios_19 -> Dataset con el listado de TODOS los beneficiarios de 2019.
2. Estados_Beneficiarios_2019_2022 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario -> Emparejamiento entre ambos listados de beneficiarios
4. Dataset_Inegi -> Catálogo obtenido de Inegi

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text) #Remove what is in between - -
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura y limpieza de datos

### 1.1 Beneficiarios 2019

Obtenemos el listado único de localidades y listado único de municipios con sus claves.

In [3]:
beneficiarios_19 = pd.read_csv('../../data/productores_beneficiarios 2019-2022/fertilizantes_2019.csv', encoding='utf-8', skiprows=0)

In [4]:
beneficiarios_19.shape

(278550, 16)

In [5]:
beneficiarios_19['ENTIDAD'].unique()

array(['GUERRERO', 'NACIONAL', nan], dtype=object)

In [6]:
beneficiarios_19.dropna(inplace=True)

In [7]:
# Obtenemos las localidades únicas en el dataset.
Localidades_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO', 'LOCALIDAD']]
Localidades_19 = Localidades_19.drop_duplicates()


In [8]:
Localidades_19['ENTIDAD_c_benef'] = Localidades_19['ENTIDAD'].apply(clean_text)
Localidades_19['MUNICIPIO_c_benef'] = Localidades_19['MUNICIPIO'].apply(clean_text)
Localidades_19['LOCALIDAD_c_benef'] = Localidades_19['LOCALIDAD'].apply(clean_text)

In [9]:
# Creamos las dos keys de beneficiarios 2019
Localidades_19['KEY_benef_mun'] = Localidades_19['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_19[
    'MUNICIPIO_c_benef'].astype(str)
Localidades_19['KEY_benef_loc'] = Localidades_19['ENTIDAD_c_benef'].astype(str) + '-' + Localidades_19[
    'MUNICIPIO_c_benef'].astype(str) + '-' + Localidades_19['LOCALIDAD_c_benef'].astype(str)

In [10]:
Localidades_19.shape

(4577, 8)

In [11]:
# Obtenemos las localidades únicas en el dataset.
Municipios_19 = beneficiarios_19[['ENTIDAD', 'MUNICIPIO']]
Municipios_19 = Municipios_19.drop_duplicates()

In [12]:
# Estandarizamos la limpieza de los datos
Municipios_19['ENTIDAD_c_benef'] = Municipios_19['ENTIDAD'].apply(clean_text)
Municipios_19['MUNICIPIO_c_benef'] = Municipios_19['MUNICIPIO'].apply(clean_text)

In [13]:
# Creamos las dos keys de beneficiarios 
Municipios_19['KEY_benef_mun'] = Municipios_19['ENTIDAD_c_benef'].astype(str) + '-' + Municipios_19[
    'MUNICIPIO_c_benef'].astype(str)

In [14]:
Municipios_19.shape

(82, 5)

In [15]:
Municipios_19.columns

Index(['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef',
       'KEY_benef_mun'],
      dtype='object')

### 1.2 INEGI 2019

Obtener listado único de municipios y listado único de localidades de inegi de 2019 con sus claves.

In [16]:
path_dataset_inegi_2019 = '../../data/dataset_inegi_clean_2019.csv'
#dataset_inegi_clean = pd.read_csv(path_dataset_inegi_2022, encoding='utf-8', dtype={'CVE_ENT': str, 'CVE_MUN': str, 'CVE_LOC': str})
dataset_inegi_2019 = pd.read_csv(path_dataset_inegi_2019)

### 1.2.1 INEGI 2019 Municipios únicos para cada año.

In [17]:
dataset_inegi_2019['KEY_inegi_municipio'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) 
dataset_inegi_2019['KEY_inegi_localidad'] = dataset_inegi_2019['Entidad_c_inegi'].astype(str) + '-' + dataset_inegi_2019[
    'Municipio_c_inegi'].astype(str) + '-' + dataset_inegi_2019['Localidad_c_inegi'].astype(str)

In [18]:
INEGI_UNIQUEMUN_2019 = dataset_inegi_2019.drop(columns=["CVE_LOC", "Localidad_inegi", "Localidad_c_inegi", "KEY_inegi_localidad"])

INEGI_UNIQUEMUN_2019 = INEGI_UNIQUEMUN_2019.drop_duplicates()

In [19]:
INEGI_UNIQUEMUN_2019.shape

(2465, 7)

In [20]:
INEGI_UNIQUEMUN_2019.columns

Index(['CVE_ENT', 'Entidad_inegi', 'CVE_MUN', 'Municipio_inegi',
       'Entidad_c_inegi', 'Municipio_c_inegi', 'KEY_inegi_municipio'],
      dtype='object')

### 1.2.1 INEGI 2019 Localidades únicas para cada año.

In [21]:
INEGI_UNIQUELOC_2019 = dataset_inegi_2019

INEGI_UNIQUELOC_2019 = INEGI_UNIQUELOC_2019.drop_duplicates()

In [22]:
INEGI_UNIQUELOC_2019.shape

(299570, 11)

# 2. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2019

El objetivo de esta sección es crear dos diccionarios de códigos según BENEFICIARIOS-MUNICIPIOS_INEGI y otro BENEFICIARIOS-LOCALIDADES_INEGI, para cada uno de los estados encontrados en inegi_2019, en este caso el número de keys es menor con lo cual no nos hará falta dividir por estado.

Para ello haremos un Left join entre Localidades_19 y el dataset de INEGI correspondiente.

In [23]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 85 y 100 de coincidencia
def fuzzy_merge_benef2019_2022(df_benef, df_inegi, key1, key2, threshold=85, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_inegi[key2].tolist()
    
    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_benef[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))


    # Crear una columna con las mejores coincidencias
    df_benef['best_match'] = [match[0] if match else None for match in matches]
    
    df_benef['match_score'] = [match[1] if match else None for match in matches]
    

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_benef, df_inegi, left_on='best_match', right_on=key2, how='left',
                         suffixes=('_benef', '_inegi'))
    
    return df_merged

In [24]:
diccionario_MUN_19 = fuzzy_merge_benef2019_2022(Municipios_19, INEGI_UNIQUEMUN_2019, 'KEY_benef_mun', 'KEY_inegi_municipio')
diccionario_MUN_19.shape

(82, 14)

In [25]:
diccionario_MUN_19.drop(columns=['ENTIDAD', 'MUNICIPIO', 'ENTIDAD_c_benef', 'MUNICIPIO_c_benef', 'Entidad_c_inegi',
       'Municipio_c_inegi'], inplace=True)

In [26]:
diccionario_MUN_19.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19.csv', index=False)

In [ ]:
diccionario_MUN_19_simple = pd.read_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/diccionario_MUN_19.csv')

In [27]:
nan_rows = beneficiarios_19[beneficiarios_19.isna().any(axis=1)]

In [28]:
nan_rows

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA


In [29]:
# Assuming your DataFrame is named df
filtered_df = beneficiarios_19[beneficiarios_19['ENTIDAD'] == 'NACIONAL']

In [30]:
filtered_df

,BENEFICIARIO,ZONA,ENTIDAD,MUNICIPIO,LOCALIDAD,ESTRATIFICACIÓN,PROGRAMA,COMPONENTE,SUBCOMPONENTE,PRODUCTO,FECHA,MONTO FEDERAL,APOYO,ACTIVIDAD,ESLABÓN,CICLO AGRÍCOLA
14,SANCHEZ RODRIGUEZ MARCELO,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,25/06/2019,2430.55,Fertilizantes,AGRARIA,AGRARIA,PV2019
96,GARCIA NAVA MARIA,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,28/06/2019,2430.55,Fertilizantes,AGRARIA,AGRARIA,PV2019
115,ROGEL ABELAR ELIZABETH,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,24/06/2019,239.00,Fertilizantes,AGRARIA,AGRARIA,PV2019
152,DEGANTE ALDAY ESTHER,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,07/07/2019,2430.55,Fertilizantes,AGRARIA,AGRARIA,PV2019
339,HERNANDEZ MORALES BERNARDO,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,28/06/2019,2191.55,Fertilizantes,AGRARIA,AGRARIA,PV2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278532,CUEVAS GARCIA SULPICIO,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,06/08/2019,239.00,Fertilizantes,AGRARIA,AGRARIA,PV2019
278533,CARRILLO VALENTE ISMAEL,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,15/07/2019,6574.65,Fertilizantes,AGRARIA,AGRARIA,PV2019
278538,BELLO ARIZMENDI MARINO,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,01/07/2019,4861.10,Fertilizantes,AGRARIA,AGRARIA,PV2019
278539,BRUNO GUERRERO HIPOLITO,NACIONAL,NACIONAL,NACIONAL,NACIONAL,SC-CONAPO,FERTILIZANTES,FERTILIZANTES,FERTILIZANTES,Cultivo de maíz grano,13/09/2019,6033.30,Fertilizantes,AGRARIA,AGRARIA,PV2019


# 3. Join por entidad (Divide y Vencerás)

### 3.1.1 Join de localidades - PUEBLA

In [31]:
Localidades_19_PUEBLA = Localidades_19[Localidades_19["ENTIDAD"] == "PUEBLA"]

dataset_inegi_uniqueloc_PUEBLA = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Puebla']

In [32]:
Localidades_19_PUEBLA = fuzzy_merge_benef2019_2022(Localidades_19_PUEBLA, dataset_inegi_uniqueloc_PUEBLA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_PUEBLA.shape

(0, 21)

### 3.2.2 Join de localidades - CHIAPAS

In [33]:
Localidades_19_CHIAPAS = Localidades_19[Localidades_19["ENTIDAD"] == "CHIAPAS"]

dataset_inegi_uniqueloc_CHIAPAS = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Chiapas']

In [34]:
Localidades_19_CHIAPAS = fuzzy_merge_benef2019_2022(Localidades_19_CHIAPAS, dataset_inegi_uniqueloc_CHIAPAS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_CHIAPAS.shape

(0, 21)

### 3.2.3 Join de localidades - DURANGO

In [35]:
Localidades_19_DURANGO = Localidades_19[Localidades_19["ENTIDAD"] == "DURANGO"]

dataset_inegi_uniqueloc_DURANGO = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Durango']

In [36]:
Localidades_19_DURANGO = fuzzy_merge_benef2019_2022(Localidades_19_DURANGO, dataset_inegi_uniqueloc_DURANGO, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_DURANGO.shape

(0, 21)

### 3.2.4 Join de localidades - MORELOS

In [37]:
Localidades_19_MORELOS = Localidades_19[Localidades_19["ENTIDAD"] == "MORELOS"]

dataset_inegi_uniqueloc_MORELOS = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Morelos']

In [38]:
Localidades_19_MORELOS = fuzzy_merge_benef2019_2022(Localidades_19_MORELOS, dataset_inegi_uniqueloc_MORELOS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_MORELOS.shape

(0, 21)

### 3.2.5 Join de localidades - GUERRERO

In [39]:
Localidades_19_GUERRERO = Localidades_19[Localidades_19["ENTIDAD"] == "GUERRERO"]

dataset_inegi_uniqueloc_GUERRERO = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Guerrero']

In [40]:
Localidades_19_GUERRERO = fuzzy_merge_benef2019_2022(Localidades_19_GUERRERO, dataset_inegi_uniqueloc_GUERRERO, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_GUERRERO.shape

### 3.2.6 Join de localidades - NAYARIT

In [ ]:
Localidades_19_NAYARIT = Localidades_19[Localidades_19["ENTIDAD"] == "NAYARIT"]

dataset_inegi_uniqueloc_NAYARIT = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Nayarit']

In [ ]:
Localidades_19_NAYARIT = fuzzy_merge_benef2019_2022(Localidades_19_NAYARIT, dataset_inegi_uniqueloc_NAYARIT, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_NAYARIT.shape

(0, 21)

### 3.2.7 Join de localidades - OAXACA

In [ ]:
Localidades_19_OAXACA = Localidades_19[Localidades_19["ENTIDAD"] == "OAXACA"]

dataset_inegi_uniqueloc_OAXACA = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Oaxaca']

In [ ]:
Localidades_19_OAXACA = fuzzy_merge_benef2019_2022(Localidades_19_OAXACA, dataset_inegi_uniqueloc_OAXACA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_OAXACA.shape

(0, 21)

### 3.2.8 Join de localidades - TLAXCALA

In [ ]:
Localidades_19_TLAXCALA = Localidades_19[Localidades_19["ENTIDAD"] == "TLAXCALA"]

dataset_inegi_uniqueloc_TLAXCALA = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Tlaxcala']

In [ ]:
Localidades_19_TLAXCALA = fuzzy_merge_benef2019_2022(Localidades_19_TLAXCALA, dataset_inegi_uniqueloc_TLAXCALA, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_TLAXCALA.shape

(0, 21)

### 3.2.9 Join de localidades - ZACATECAS

In [ ]:
Localidades_19_ZACATECAS = Localidades_19[Localidades_19["ENTIDAD"] == "ZACATECAS"]

dataset_inegi_uniqueloc_ZACATECAS = INEGI_UNIQUELOC_2019[INEGI_UNIQUELOC_2019['Entidad_inegi'] == 'Zacatecas']

In [ ]:
Localidades_19_ZACATECAS = fuzzy_merge_benef2019_2022(Localidades_19_ZACATECAS, dataset_inegi_uniqueloc_ZACATECAS, 'KEY_benef_loc', 'KEY_inegi_localidad')
Localidades_19_ZACATECAS.shape

(0, 21)

In [ ]:
Localidades_19_GUERRERO.to_csv('../../data/productores_beneficiarios 2019-2022/diccionarios_E3/Localidades_19_GUERRERO.csv', index=False)